In [ ]:
#%pip install boto3 protobuf gtfs-realtime-bindings


Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd
from google.transit import gtfs_realtime_pb2

bucket = "bus-delay-forecast"
prefix = "gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/"

s3 = boto3.client("s3")

resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
keys = [o["Key"] for o in resp.get("Contents", []) if o["Key"].endswith(".pb")]
keys.sort()

print("Using:", keys[0])

pb = s3.get_object(Bucket=bucket, Key=keys[0])["Body"].read()

feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(pb)

rows = []
for ent in feed.entity:
    if not ent.HasField("trip_update"):
        continue
    tu = ent.trip_update
    for stu in tu.stop_time_update:
        rows.append({
            "trip_id": tu.trip.trip_id,
            "route_id": tu.trip.route_id,
            "stop_id": stu.stop_id,
            "stop_sequence": stu.stop_sequence if stu.HasField("stop_sequence") else None,
            "delay_sec": stu.arrival.delay if stu.HasField("arrival") else None
        })

df = pd.DataFrame(rows)
df.head(50)


Using: gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040810.pb


,trip_id,route_id,stop_id,stop_sequence,delay_sec
0,14827490,,31,29,43.0
1,14827491,6612,31,29,-34.0
2,14827492,6612,12040,19,-191.0
3,14827492,6612,10542,20,-199.0
4,14827492,6612,69,21,-212.0
5,14827492,6612,70,22,-207.0
6,14827492,6612,72,23,-214.0
7,14827492,6612,73,24,-256.0
8,14827492,6612,75,25,-220.0
9,14827492,6612,11881,26,-197.0


In [6]:
import boto3

bucket = "bus-delay-forecast"
prefix = "gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/"

s3 = boto3.client("s3")

resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
keys = sorted([o["Key"] for o in resp.get("Contents", []) if o["Key"].endswith(".pb")])

print("Found:", len(keys))
print("First 5 keys:")
for k in keys[:5]:
    print(k)

# pick one that definitely exists
key = keys[0]
print("\nUsing key:", key)

pb_bytes = s3.get_object(Bucket=bucket, Key=key)["Body"].read()
print("Downloaded bytes:", len(pb_bytes))


Found: 60
First 5 keys:
gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040810.pb
gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040870.pb
gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040930.pb
gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040990.pb
gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770041050.pb

Using key: gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040810.pb
Downloaded bytes: 661670


In [7]:
import boto3

bucket = "bus-delay-forecast"
prefix = "gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/"

s3 = boto3.client("s3")

resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
keys = sorted(o["Key"] for o in resp["Contents"] if o["Key"].endswith(".pb"))

print("Found:", len(keys))
print("Example key:", keys[0])

pb_bytes = s3.get_object(Bucket=bucket, Key=keys[0])["Body"].read()
print("Bytes:", len(pb_bytes))


Found: 60
Example key: gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040810.pb
Bytes: 661670


In [8]:
from google.transit import gtfs_realtime_pb2

feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(pb_bytes)

# Print the first full TripUpdate
for ent in feed.entity:
    if ent.HasField("trip_update"):
        print(ent.trip_update)
        break

trip {
  trip_id: "14827490"
  start_date: "20260202"
  schedule_relationship: SCHEDULED
  direction_id: 0
}
stop_time_update {
  stop_sequence: 29
  arrival {
    delay: 43
    time: 1770039216
  }
  departure {
    delay: 43
    time: 1770039216
  }
  stop_id: "31"
  schedule_relationship: SCHEDULED
}
vehicle {
}



In [9]:
print(feed.header)

gtfs_realtime_version: "2.0"
incrementality: FULL_DATASET
timestamp: 1770040801



# Testing with monday - 8 am

In [3]:
# Test with 14 hours
import re, io
import boto3
import pandas as pd
from google.transit import gtfs_realtime_pb2

bucket = "bus-delay-forecast"
in_prefix = "gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/"
out_key = "processed/gtfs_rt/trip_updates_parquet/year=2026/month=02/day=02/hour=14/part-000.parquet"

s3 = boto3.client("s3")
TS_RE = re.compile(r"ts=(\d+)\.pb$")

def parse_tripupdates(pb_bytes: bytes, default_feed_ts=None):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(pb_bytes)
    feed_ts = feed.header.timestamp if feed.header and feed.header.timestamp else default_feed_ts

    rows = []
    for ent in feed.entity:
        if not ent.HasField("trip_update"):
            continue
        tu = ent.trip_update
        trip_id = tu.trip.trip_id if tu.trip.trip_id else None
        route_id = tu.trip.route_id if tu.trip.route_id else None

        for stu in tu.stop_time_update:
            delay_sec = None
            if stu.HasField("arrival") and stu.arrival.HasField("delay"):
                delay_sec = stu.arrival.delay
            elif stu.HasField("departure") and stu.departure.HasField("delay"):
                delay_sec = stu.departure.delay

            rows.append({
                "feed_timestamp": feed_ts,
                "trip_id": trip_id,
                "route_id": route_id,
                "stop_id": stu.stop_id if stu.stop_id else None,
                "stop_sequence": stu.stop_sequence if stu.HasField("stop_sequence") else None,
                "delay_sec": delay_sec,
            })
    return rows

# 1) list pb files in hour=14
resp = s3.list_objects_v2(Bucket=bucket, Prefix=in_prefix)
keys = sorted([o["Key"] for o in resp.get("Contents", []) if o["Key"].endswith(".pb")])
print("PB files found:", len(keys), "Example:", keys[0])

# 2) parse a manageable number first (e.g., first 10 files)
all_rows = []
for k in keys[:10]:
    m = TS_RE.search(k)
    ts_from_name = int(m.group(1)) if m else None
    pb = s3.get_object(Bucket=bucket, Key=k)["Body"].read()
    all_rows.extend(parse_tripupdates(pb, default_feed_ts=ts_from_name))

df = pd.DataFrame(all_rows)

# 3) add handy time features
df["timestamp"] = pd.to_datetime(df["feed_timestamp"], unit="s", utc=True)
df["hour"] = df["timestamp"].dt.hour
df["day_of_week"] = df["timestamp"].dt.dayofweek

print(df.shape)
df.head()


PB files found: 60 Example: gtfs_rt/trip_updates/year=2026/month=02/day=02/hour=14/ts=1770040810.pb
(142906, 9)


,feed_timestamp,trip_id,route_id,stop_id,stop_sequence,delay_sec,timestamp,hour,day_of_week
0,1770040801,14827490,None,31,29,43.0,2026-02-02 14:00:01+00:00,14,0
1,1770040801,14827491,6612,31,29,-34.0,2026-02-02 14:00:01+00:00,14,0
2,1770040801,14827492,6612,12040,19,-191.0,2026-02-02 14:00:01+00:00,14,0
3,1770040801,14827492,6612,10542,20,-199.0,2026-02-02 14:00:01+00:00,14,0
4,1770040801,14827492,6612,69,21,-212.0,2026-02-02 14:00:01+00:00,14,0


In [4]:
import io
import boto3

bucket = "bus-delay-forecast"
out_key = (
    "processed/gtfs_rt/trip_updates_parquet/"
    "year=2026/month=02/day=02/hour=14/part-000.parquet"
)

s3 = boto3.client("s3")

buf = io.BytesIO()
df.to_parquet(buf, index=False)
buf.seek(0)

s3.put_object(
    Bucket=bucket,
    Key=out_key,
    Body=buf.getvalue()
)

print("Wrote:", f"s3://{bucket}/{out_key}")



Wrote: s3://bus-delay-forecast/processed/gtfs_rt/trip_updates_parquet/year=2026/month=02/day=02/hour=14/part-000.parquet


In [ ]:
import pandas as pd

df_public = pd.read_parquet(
    "s3://bus-delay-forecast/processed/gtfs_rt/trip_updates_parquet/year=2026/month=02/day=02/hour=14/part-000.parquet",
    storage_options={"anon": True},
)

df_public.head()


TypeError: ClientArgsCreator.compute_endpoint_resolver_builtin_defaults() missing 2 required positional arguments: 'credentials' and 'account_id_endpoint_mode'

In [1]:
from datetime import date, timedelta

# Yesterday (Friday)
end_date = date.today() - timedelta(days=1)

# Monday of that week
start_date = end_date - timedelta(days=end_date.weekday())

print(start_date, "→", end_date)


2026-02-02 → 2026-02-06


# Starting to build the entire data

In [2]:
import boto3
import pandas as pd
import io
import re
from google.transit import gtfs_realtime_pb2


In [3]:
s3 = boto3.client("s3")
bucket = "bus-delay-forecast"


In [7]:
TS_RE = re.compile(r"ts=(\d+)\.pb$")

def parse_tripupdates(pb_bytes, default_feed_ts=None):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(pb_bytes)

    feed_ts = (
        feed.header.timestamp
        if feed.header and feed.header.timestamp
        else default_feed_ts
    )

    rows = []
    for ent in feed.entity:
        if not ent.HasField("trip_update"):
            continue

        tu = ent.trip_update
        trip = tu.trip

        for stu in tu.stop_time_update:
            delay_sec = None
            if stu.HasField("arrival") and stu.arrival.HasField("delay"):
                delay_sec = stu.arrival.delay
            elif stu.HasField("departure") and stu.departure.HasField("delay"):
                delay_sec = stu.departure.delay

            rows.append({
                "feed_timestamp": feed_ts,
                "trip_id": trip.trip_id or None,
                "route_id": trip.route_id or None,
                "stop_id": stu.stop_id or None,
                "stop_sequence": stu.stop_sequence if stu.HasField("stop_sequence") else None,
                "delay_sec": delay_sec,
            })

    return rows


In [8]:
# FAST Mon–Fri build to S3 Parquet (parallel downloads + safe pagination)
# ---------------------------------------------------------------
# What it does:
# - Builds test dataset for Monday→Friday (yesterday’s week)
# - Discovers available hour=XX partitions automatically per day
# - Downloads + parses .pb files in parallel (ThreadPoolExecutor)
# - Writes one Parquet per (day, hour) into processed/.../split=test/
#
# Requirements:
#   pip install boto3 pandas pyarrow gtfs-realtime-bindings protobuf
#
# Notes:
# - Uses Feed header timestamp if present; falls back to ts=... from filename.
# - S3 "folders" don't need to exist beforehand.
# ---------------------------------------------------------------

from datetime import date, timedelta
import io
import re
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

import boto3
import pandas as pd
from google.transit import gtfs_realtime_pb2


# -------------------------
# 0) Date window: Mon→Fri (yesterday)
# -------------------------
end_date = date.today() - timedelta(days=1)                 # yesterday (Fri)
start_date = end_date - timedelta(days=end_date.weekday())  # Monday of same week
print("Test window:", start_date, "→", end_date)


# -------------------------
# 1) Config
# -------------------------
bucket = "bus-delay-forecast"
RAW_BASE = "gtfs_rt/trip_updates"
OUT_BASE = "processed/gtfs_rt/trip_updates_parquet/split=test"

# Tune these:
MAX_WORKERS = 24          # 12–32 is usually good; reduce if you hit throttling
MAX_FILES_PER_HOUR = None # set to e.g. 20 for a quick dry run; None = all
WRITE_MODE = "overwrite"  # "overwrite" always writes part-000.parquet (safe reruns)

TS_RE = re.compile(r"ts=(\d+)\.pb$")

s3 = boto3.client("s3")


# -------------------------
# 2) Helpers
# -------------------------
def list_pb_keys(bucket: str, prefix: str) -> list[str]:
    """List all .pb keys under prefix (handles pagination)."""
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            k = obj["Key"]
            if k.endswith(".pb"):
                keys.append(k)
    keys.sort()
    return keys


def list_hour_prefixes(bucket: str, day_prefix: str) -> list[str]:
    """List hour=XX/ prefixes for a given day prefix."""
    # Delimiter listing returns folders in CommonPrefixes
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=day_prefix, Delimiter="/")
    return [p["Prefix"] for p in resp.get("CommonPrefixes", [])]


def parse_tripupdates(pb_bytes: bytes, default_feed_ts=None) -> list[dict]:
    """Flatten TripUpdates: 1 row per stop_time_update."""
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(pb_bytes)

    feed_ts = (feed.header.timestamp if feed.header and feed.header.timestamp else default_feed_ts)

    rows = []
    for ent in feed.entity:
        if not ent.HasField("trip_update"):
            continue

        tu = ent.trip_update
        trip = tu.trip

        trip_id = trip.trip_id or None
        route_id = trip.route_id or None

        for stu in tu.stop_time_update:
            # delay preference: arrival.delay else departure.delay
            delay_sec = None
            if stu.HasField("arrival") and stu.arrival.HasField("delay"):
                delay_sec = stu.arrival.delay
            elif stu.HasField("departure") and stu.departure.HasField("delay"):
                delay_sec = stu.departure.delay

            rows.append({
                "feed_timestamp": feed_ts,
                "trip_id": trip_id,
                "route_id": route_id,
                "stop_id": (stu.stop_id or None),
                "stop_sequence": (stu.stop_sequence if stu.HasField("stop_sequence") else None),
                "delay_sec": delay_sec,
            })

    return rows


def fetch_and_parse(key: str) -> list[dict]:
    """Download one pb object and parse rows."""
    m = TS_RE.search(key)
    ts_from_name = int(m.group(1)) if m else None

    pb = s3.get_object(Bucket=bucket, Key=key)["Body"].read()
    return parse_tripupdates(pb, default_feed_ts=ts_from_name)


def write_parquet_to_s3(df: pd.DataFrame, out_key: str):
    """Write dataframe as Parquet to S3."""
    buf = io.BytesIO()
    df.to_parquet(buf, index=False)
    buf.seek(0)
    s3.put_object(Bucket=bucket, Key=out_key, Body=buf.getvalue())


# -------------------------
# 3) Main loop: Mon→Fri, all hours, fast parallel parse
# -------------------------
current = start_date
total_hours_written = 0

while current <= end_date:
    y, m, d = current.year, current.month, current.day
    day_prefix = f"{RAW_BASE}/year={y}/month={m:02d}/day={d:02d}/"

    hour_prefixes = list_hour_prefixes(bucket, day_prefix)

    print(f"\n📅 {current} | hours found: {len(hour_prefixes)} | prefix: {day_prefix}")

    for hour_prefix in hour_prefixes:
        # hour_prefix looks like ".../day=DD/hour=14/"
        hour = hour_prefix.split("hour=")[-1].rstrip("/")
        t0 = time.time()

        keys = list_pb_keys(bucket, hour_prefix)
        if not keys:
            print(f"  [SKIP] hour={hour} (no .pb files)")
            continue

        if MAX_FILES_PER_HOUR is not None:
            keys = keys[:MAX_FILES_PER_HOUR]

        print(f"  ⏳ hour={hour} files={len(keys)} (parallel workers={MAX_WORKERS})")

        all_rows = []
        # Parallel download+parse (I/O bound → threads help a lot)
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
            futures = [ex.submit(fetch_and_parse, k) for k in keys]
            for i, fut in enumerate(as_completed(futures), 1):
                all_rows.extend(fut.result())
                if i % 10 == 0 or i == len(keys):
                    print(f"    ...parsed {i}/{len(keys)} files")

        if not all_rows:
            print(f"  [SKIP] hour={hour} (no trip_update rows)")
            continue

        df = pd.DataFrame(all_rows)

        # Add time features (UTC + Vancouver)
        df["timestamp"] = pd.to_datetime(df["feed_timestamp"], unit="s", utc=True)
        df["timestamp_pt"] = df["timestamp"].dt.tz_convert("America/Vancouver")
        df["hour_pt"] = df["timestamp_pt"].dt.hour
        df["day_of_week_pt"] = df["timestamp_pt"].dt.dayofweek

        out_key = f"{OUT_BASE}/year={y}/month={m:02d}/day={d:02d}/hour={hour}/part-000.parquet"

        write_parquet_to_s3(df, out_key)

        total_hours_written += 1
        print(
            f"  ✅ WROTE hour={hour} rows={len(df):,} -> s3://{bucket}/{out_key} "
            f"({time.time()-t0:.1f}s)"
        )

    current += timedelta(days=1)

print(f"\nDONE ✅ Total hour partitions written: {total_hours_written}")


Test window: 2026-02-02 → 2026-02-07

📅 2026-02-02 | hours found: 6 | prefix: gtfs_rt/trip_updates/year=2026/month=02/day=02/
  ⏳ hour=07 files=1 (parallel workers=24)
    ...parsed 1/1 files
  ✅ WROTE hour=07 rows=5,869 -> s3://bus-delay-forecast/processed/gtfs_rt/trip_updates_parquet/split=test/year=2026/month=02/day=02/hour=07/part-000.parquet (1.2s)
  ⏳ hour=13 files=60 (parallel workers=24)
    ...parsed 10/60 files
    ...parsed 20/60 files
    ...parsed 30/60 files
    ...parsed 40/60 files
    ...parsed 50/60 files
    ...parsed 60/60 files
  ✅ WROTE hour=13 rows=519,919 -> s3://bus-delay-forecast/processed/gtfs_rt/trip_updates_parquet/split=test/year=2026/month=02/day=02/hour=13/part-000.parquet (3.3s)
  ⏳ hour=14 files=60 (parallel workers=24)
    ...parsed 10/60 files
    ...parsed 20/60 files
    ...parsed 30/60 files
    ...parsed 40/60 files
    ...parsed 50/60 files
    ...parsed 60/60 files
  ✅ WROTE hour=14 rows=1,047,331 -> s3://bus-delay-forecast/processed/gtfs_rt/tr

In [11]:
import pandas as pd

df_test = pd.read_parquet(
    "s3://bus-delay-forecast/processed/gtfs_rt/trip_updates_parquet/split=test/"
)

df_test.shape
df_test.head()




,feed_timestamp,trip_id,route_id,stop_id,stop_sequence,delay_sec,timestamp,timestamp_pt,hour_pt,day_of_week_pt,year,month,day,hour
0,1770016800,14827965,6612,31,29,-18.0,2026-02-02 07:20:00+00:00,2026-02-01 23:20:00-08:00,23,6,2026,2,2,7
1,1770016800,14827966,6612,1276,9,57.0,2026-02-02 07:20:00+00:00,2026-02-01 23:20:00-08:00,23,6,2026,2,2,7
2,1770016800,14827966,6612,1277,10,49.0,2026-02-02 07:20:00+00:00,2026-02-01 23:20:00-08:00,23,6,2026,2,2,7
3,1770016800,14827966,6612,1279,11,20.0,2026-02-02 07:20:00+00:00,2026-02-01 23:20:00-08:00,23,6,2026,2,2,7
4,1770016800,14827966,6612,56,12,10.0,2026-02-02 07:20:00+00:00,2026-02-01 23:20:00-08:00,23,6,2026,2,2,7
